<a href="https://colab.research.google.com/github/kaanbiryol/tensorflow-colab-pepsi/blob/master/pepsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Packages**

In [0]:
%cd
  
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive tensorboardcolab

!pip install -q pycocotools


/root


In [0]:
%cd 
!rm -rf models

/root


In [0]:
%cd ~/../content
!ls
!rm -rf data

/content
adc.json  csv  data  fine_tuned_model  tfrecord  training


In [0]:
%cd ~/../content
!ls
!rm -rf training
!rm -rf fine_tuned_model

/content
adc.json  csv  fine_tuned_model  tfrecord  training


# **Create Directories**

In [0]:
%cd ~/../content
!rm -rf sample_data
!ls
!mkdir data csv tfrecord training
%cd data
!mkdir training test
!ls
%cd ~/../content
!ls

#%cd training
#!echo "item {\n id: 1\n name: 'pepsi_can'\n} item {\n id: 2\n name: 'pepsi_bottle'\n} " > labels.pbtxt
#!cat labels.pbtxt

/content
adc.json  csv  tfrecord
mkdir: cannot create directory ‘csv’: File exists
mkdir: cannot create directory ‘tfrecord’: File exists
/content/data
test  training
/content
adc.json  csv  data  tfrecord  training


# **Generate protoc files**

In [0]:
%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/root/models/research
......................
----------------------------------------------------------------------
Ran 22 tests in 0.116s

OK


# **TODO: Generate script to upload folder or upload files from URL **
For now just upload them from files section

# **XML To CSV**

In [0]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import argparse

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    
    image_path = os.path.join("/content/data/training")
    if not os.path.isdir(image_path):
        print('please provide a valid directory')
        return
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv("/content/csv/training", index=None)
    print('Successfully converted training xml to csv.')

    image_path = os.path.join("/content/data/test")
    if not os.path.isdir(image_path):
        print('please provide a valid directory')
        return
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv("/content/csv/test", index=None)
    print('Successfully converted test xml to csv.')
    
main()


Successfully converted training xml to csv.
Successfully converted test xml to csv.


# **Generate .tfrecord**

In [0]:
"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=training.csv  --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import argparse

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'pepsi_can':
        return 1
    elif row_label == 'pepsi_bottle':
        return 2
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    
    path = os.path.join("/content/data/training")
    if not os.path.isdir(path):
        print('please provide a valid directory')
        return
    
    writer = tf.python_io.TFRecordWriter("/content/tfrecord/training.record")
    examples = pd.read_csv("/content/csv/training")
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_name = os.path.join("/content/tfrecord/training.record")
    print('Successfully created the TFRecords: {}'.format(output_name))

    
    path = os.path.join("/content/data/test")
    if not os.path.isdir(path):
        print('please provide a valid directory')
        return
    
    writer = tf.python_io.TFRecordWriter("/content/tfrecord/test.record")
    examples = pd.read_csv("/content/csv/test")
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_name = os.path.join("/content/tfrecord/test.record")
    print('Successfully created the TFRecords: {}'.format(output_name))

if __name__ == '__main__':
    tf.app.run()
    

Successfully created the TFRecords: /content/tfrecord/training.record
Successfully created the TFRecords: /content/tfrecord/test.record


SystemExit: ignored

# Download Pretrained Model

In [0]:
%cd ~/../content

import os
import shutil
import glob
import urllib
import tarfile
#ssd_mobilenet_v1_coco_11_06_2017
#faster_rcnn_inception_v2_coco_2018_01_28
MODEL = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'training/pre_trained_model'

if not (os.path.exists(MODEL_FILE)):
  opener = urllib.URLopener()
  opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content


# **Edit pipeline.config**

In [0]:
import re

#filename = '/content/datalab/pretrained_model/pipeline.config'
filename = '/root/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'
with open(filename) as f:
  s = f.read()
with open(filename, 'w') as f:
  s = re.sub('num_classes: 90', 'num_classes: 2', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/model.ckpt', '/content/training/model.ckpt', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_train.record-\?\?\?\?\?-of-00100', '/content/tfrecord/training.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_val.record-\?\?\?\?\?-of-00010', '/content/tfrecord/test.record', s)
  s = re.sub('PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt', '/content/training/labels.pbtxt', s)
  s = re.sub('max_evals: 10', ' ', s)
  f.write(s)

!head -2000 /root/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config
#!tail -n 50 /root/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_pets.config

In [0]:
%cd ~/../content/training
!cat ssd_mobilenet_v2_coco.config


# **Train model**

In [0]:
%cd ~/../content/training

#/root/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
!python ~/models/research/object_detection/model_main.py \
    --pipeline_config_path=/content/training/ssd_mobilenet_v1_pets.config \
    --model_dir=/content/training/trained_model \
    --logtostderr \
   #--num_train_steps=2000

/content/training
/root/models/research/object_detection/utils/visualization_utils.py:26: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/root/models/research/object_detection/model_main.py", line 26, in <module>
    from object_detection import model_lib
  File "/root/models/research/object_detection/model_lib.py", line 27, in <module>
    from object_detection import eval_util
  File "/root/models/research/object_detection/eval_util.py", line 27, in <module>
    from object_detection.metrics import coco_evaluation
  File "/root/models/research/object_detection/metrics/coco_evaluation.py", line 20, in <module>
    from object_detection.metrics import coco_tools
  File "/root/models/research/object_dete

# **(Optional for Tensorboard)** Zip and Download Training Output

In [0]:
%cd ~/../

#@title Zip and Download
filename = "training" #@param {type:"string"}
folders_or_files_to_save = "content/training/trained_model" #@param {type:"string"}
from google.colab import files
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

def save_file_to_drive(name, path):
    file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
    }

    media = MediaFileUpload(path, 
                  mimetype='application/octet-stream',
                  resumable=True)

    created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created


extension_zip = ".zip"

zip_file = filename + extension_zip

# !rm -rf $zip_file
!zip -r $zip_file {folders_or_files_to_save} # FOLDERS TO SAVE INTO ZIP FILE

auth.authenticate_user()
drive_service = build('drive', 'v3')

destination_name = zip_file
path_to_file = zip_file
save_file_to_drive(destination_name, path_to_file)

/
  adding: content/training/trained_model/ (stored 0%)
  adding: content/training/trained_model/model.ckpt-1310.meta (deflated 93%)
  adding: content/training/trained_model/checkpoint (deflated 72%)
  adding: content/training/trained_model/eval_0/ (stored 0%)
  adding: content/training/trained_model/eval_0/events.out.tfevents.1545312993.f6bebe64828a (deflated 20%)
  adding: content/training/trained_model/model.ckpt-872.index (deflated 72%)
  adding: content/training/trained_model/model.ckpt-872.meta (deflated 93%)
  adding: content/training/trained_model/model.ckpt-435.meta (deflated 93%)
  adding: content/training/trained_model/events.out.tfevents.1545312359.f6bebe64828a (deflated 93%)
  adding: content/training/trained_model/model.ckpt-2000.data-00000-of-00001 (deflated 7%)
  adding: content/training/trained_model/model.ckpt-1748.index (deflated 72%)
  adding: content/training/trained_model/model.ckpt-1310.data-00000-of-00001 (deflated 7%)
  adding: content/training/trained_model/mo

{u'id': u'1LzBmN7Pa_4CTHclAGYMIpT0KcnBQfx3g'}

# **Export trained model as .pb**

In [0]:
%cd ~/../content

lst = os.listdir('training/trained_model')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
last_model = sorted(lf)[-1].replace('.meta', '')

!python ~/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/training/ssd_mobilenet_v1_pets.config \
    --output_directory=fine_tuned_model \
    --trained_checkpoint_prefix=training/trained_model/$last_model


/content
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for details
117 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                       -1
-order_by                   name
-account_type_regexes       _trainable_variables
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*
-show_name_regexes          .*
-hide_name_regexes          
-account

# **Save .pb to Drive**

In [0]:
%cd ~/../
#!zip -r /content/fine_tuned_model.zip /content/fine_tuned_model

#from google.colab import files
#files.download("/content/fine_tuned_model.zip")

#@title Zip and Download
filename = "fine_tuned_model" #@param {type:"string"}
folders_or_files_to_save = "content/fine_tuned_model" #@param {type:"string"}
from google.colab import files
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

def save_file_to_drive(name, path):
    file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
    }

    media = MediaFileUpload(path, 
                  mimetype='application/octet-stream',
                  resumable=True)

    created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created


extension_zip = ".zip"

zip_file = filename + extension_zip

# !rm -rf $zip_file
!zip -r $zip_file {folders_or_files_to_save} # FOLDERS TO SAVE INTO ZIP FILE

auth.authenticate_user()
drive_service = build('drive', 'v3')

destination_name = zip_file
path_to_file = zip_file
save_file_to_drive(destination_name, path_to_file)


/
updating: content/fine_tuned_model/ (stored 0%)
updating: content/fine_tuned_model/saved_model/ (stored 0%)
updating: content/fine_tuned_model/saved_model/saved_model.pb (deflated 9%)
updating: content/fine_tuned_model/saved_model/variables/ (stored 0%)
updating: content/fine_tuned_model/checkpoint (deflated 42%)
updating: content/fine_tuned_model/frozen_inference_graph.pb (deflated 9%)
updating: content/fine_tuned_model/model.ckpt.meta (deflated 93%)
updating: content/fine_tuned_model/pipeline.config (deflated 69%)
updating: content/fine_tuned_model/model.ckpt.index (deflated 67%)
updating: content/fine_tuned_model/model.ckpt.data-00000-of-00001 (deflated 7%)
File ID: 18T3pcZTyNup8p4W4VRRfDXDQIQZ30IIj


{u'id': u'18T3pcZTyNup8p4W4VRRfDXDQIQZ30IIj'}

# **Inference**

# **Possible Errors**

**WARNING:tensorflow:Estimator's model_fn (<function model_fn at 0x7f2154614b90>) includes params argument, but params are not passed to Estimator.**

Remove checkpoint file inside /training

**No module named object_detection.**

Run protoc commands.

In [0]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()

drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
  file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

  media = MediaFileUpload(path, 
                    mimetype='application/octet-stream',
                    resumable=True)

  created = drive_service.files().create(body=file_metadata,
                                   media_body=media,
                                   fields='id').execute()

  print('File ID: {}'.format(created.get('id')))

  return created

def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            print(filepath)
            save_file_to_drive(filename, filepath)
            file_paths.append(filepath)  # Add it to the list.


get_filepaths("/content/training")


In [0]:
%cd ~/..

#from tensorboardcolab import *
#tbc=TensorBoardColab()

!zip -r /content/training_output.zip /content/training

In [0]:
from tensorboardcolab import *
tbc=TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://7247a41d.ngrok.io


In [0]:
%cd ~/../content/training
!rm -rf checkpoint

/content/training


In [0]:
from google.colab import files
files.download("/content/training_output.zip")